In [ ]:
# stdlib
import os

# os.environ["ORCHESTRA_DEPLOYMENT_TYPE"] = "remote"
# os.environ["DEV_MODE"] = "True"
# os.environ["TEST_EXTERNAL_REGISTRY"] = "k3d-registry.localhost:5800"
# os.environ["CLUSTER_HTTP_PORT_HIGH"] = "9081"
# os.environ["CLUSTER_HTTP_PORT_LOW"] = "9083"

In [ ]:
# stdlib

# third party
import pandas as pd

# syft absolute
import syft as sy
from syft import test_helpers  # noqa: F401
from syft.client.syncing import compare_clients
from syft.util.test_helpers.email_helpers import get_email_server

In [ ]:
environment = os.environ.get("ORCHESTRA_DEPLOYMENT_TYPE", "python")
high_port = os.environ.get("CLUSTER_HTTP_PORT_HIGH", "9081")
low_port = os.environ.get("CLUSTER_HTTP_PORT_LOW", "9083")

# Launch server and login

In [ ]:
server_low = sy.orchestra.launch(
    name="bigquery-low",
    server_side_type="low",
    dev_mode=True,
    n_consumers=1,
    create_producer=True,
    port=low_port,
)

server_high = sy.orchestra.launch(
    name="bigquery-high",
    server_side_type="high",
    dev_mode=True,
    n_consumers=1,
    create_producer=True,
    port=high_port,
)

In [ ]:
email_server_low, smtp_server_low = get_email_server(server_side_type="low")
email_server_high, smtp_server_high = get_email_server(server_side_type="high")

In [ ]:
low_client = server_low.login(email="info@openmined.org", password="changethis")
high_client = server_high.login(email="info@openmined.org", password="changethis")

# Sync UserCode and Requests to High Side

In [ ]:
diffs = compare_clients(
    from_client=low_client, to_client=high_client, hide_usercode=False
)

In [ ]:
# check that only requests and usercode are in the diff
assert {diff.root_diff.obj_type.__qualname__ for diff in diffs.batches} == {
    "Request",
    "UserCode",
}

In [ ]:
widget = diffs.resolve()

widget._share_all()
widget._sync_all()

# Check that request synced over to high side

In [ ]:
assert len(high_client.code.get_all()) == 2

In [ ]:
requests = high_client.requests
requests

In [ ]:
user_request = None
for request in requests:
    if "large_sample" in getattr(
        getattr(request, "code", None), "service_func_name", None
    ):
        user_request = request
assert user_request

# Execute Request

In [ ]:
job = high_client.code.large_sample(blocking=False)

In [ ]:
res = job.wait().get()

assert isinstance(res, pd.DataFrame)

# Sync job result to low side

In [ ]:
diffs = compare_clients(
    from_client=high_client, to_client=low_client, hide_usercode=False
)

In [ ]:
batch_root_strs = [x.root_diff.obj_type.__qualname__ for x in diffs.batches]
assert len(diffs.batches) == 3
assert "Job" in batch_root_strs

In [ ]:
widget = diffs.resolve()

widget._share_all()
widget._sync_all()

# Check that job was synced to low side

In [ ]:
requests = low_client.requests
requests

In [ ]:
user_request = None
for request in requests:
    if "large_sample" in getattr(
        getattr(request, "code", None), "service_func_name", None
    ):
        user_request = request

In [ ]:
assert "approved" in str(user_request.status).lower()

# Test email

In [ ]:
# Should this user be notified about approval? Right now it seems that it is not.
assert (
    len(email_server_low.get_emails_for_user(user_email="data_scientist@openmined.org"))
    == 1
)

In [ ]:
# Similarly, after syncing, should the admin be notified about anything?

# Shutdown

In [ ]:
if environment != "remote":
    server_high.land()
    server_low.land()
smtp_server_low.stop()
smtp_server_high.stop()